### Run from within cloned CLEAN repo so that it's easier
1. Clone the CLEAN repo
2. Follow the installation instruction
3. Move this file to `CLEAN/app` (should be in same folder as demo)
4. Move `EC2protein_train.csv` and `test.csv` to `CLEAN/app/data`
5. Run this notebook from the CLEAN directory

In [ ]:
from CLEAN.utils import *
ensure_dirs("data/esm_data")
ensure_dirs("data/pretrained")

### training
might be better to do this in a script if it takes a long time


In [ ]:
from CLEAN.utils import mutate_single_seq_ECs, retrive_esm1b_embedding, compute_esm_distance

train_file = "EC2protein_train"
train_fasta_file = mutate_single_seq_ECs(train_file) #get rid of duplicates

#retrieving embeddings and distance matrix needs to be done once per training dataset
retrive_esm1b_embedding(train_fasta_file)
compute_esm_distance(train_file)

In [ ]:
!./train-supconH.py --training_data EC2protein_train --model_name EC2protein_train_supconH --epoch 1500 --n_pos 9 --n_neg 30 -T 0.1

### inference on the test set

In [ ]:
csv_to_fasta("data/test.csv", "data/test.fasta")
retrive_esm1b_embedding("test")

In [ ]:
from CLEAN.infer import infer_pvalue
train_data = "EC2protein_train"
test_data = "test"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20, report_metrics=True, pretrained=True)